In [56]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import pandas as pd
from selenium.common.exceptions import NoSuchElementException #셀레니엄 예외처리용

#웹브라우저를 띄우지 않고 진행하기 위한 설정
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [73]:
def get_movie_reveiws(url, page_num=10):
    wd = webdriver.Chrome('chromedriver', options=chrome_options)
    wd.get(url)
    
    writer_list = [] #리스트는 항상 선언 먼저 해주자!
    review_list = []
    date_list = []
    
    for page_no in range(1, page_num+1):
        try:
#             print(page_num)
            page_ul = wd.find_element(By.ID, 'paging_point') #<ul> 태그로 감싸져있으니 먼저 전체를 들고 오기
            #셀레니움 기능 중 하나, 특정 텍스트의 링크를 가져오게 할 수 있음(.LINK_TEXT())
            page_a = page_ul.find_element(By.LINK_TEXT, str(page_no))
            page_a.click()
            time.sleep(1)

            writers = wd.find_elements(By.CLASS_NAME, 'writer-name')
            writer_list += [writer.text for writer in writers]

            reviews = wd.find_elements(By.CLASS_NAME, 'box-comment')
            review_list += [review.text for review in reviews]

            dates = wd.find_elements(By.CLASS_NAME, 'day')
            date_list += [date.text for date in dates]

            if page_no % 10 == 0: 
                next_button = page_ul.find_element(By.CSS_SELECTOR, '#paging_point > li > button')
                next_button.click()
                time.sleep(1)
            
        except NoSuchElementException: #다음 페이지가 없을때 break
            break
        
    movie_review_df = pd.DataFrame({"Writer":writer_list,
                                   "Review":review_list,
                                   "Date":date_list})
    return movie_review_df

In [74]:
url = 'http://www.cgv.co.kr/movies/detail-view/?midx=84949#3'
movie_review_df = get_movie_reveiws(url, 12)
movie_review_df

,Writer,Review,Date
0,mi**e1211,감동적이었음...굳,2022.01.11
1,ge**vievej,"내용은 별로였는데 스파이더맨이 좋았어요, 보면서 울컥했어요",2022.01.11
2,cr**ynam,정말 좋아서 3번이나 봤어요!,2022.01.11
3,kw**368,"4DX와 함께한 스파이더맨은 몰입감, 현장감 최고였어요!",2022.01.11
4,체다히,마블은 언제나 재밌음 몇천번이고 볼 수 있음 배우들의 연기도 외모도 너무 완벽함 마...,2022.01.11
...,...,...,...
67,aj**665,추억과 재미가 공존하는 영화,2022.01.11
68,드드드ㅡ,3번 봐도 개꿀잼 또보고싶다,2022.01.11
69,대차리토,소름돋아요 덜덜! 고마워요,2022.01.11
70,심해인,톰 홀랜드와 젠다이아의 연기는 언제 봐도 대단.,2022.01.11


### CGV 상영작 스크래핑

In [53]:
url = 'http://www.cgv.co.kr/movies/?lt=1&ft=0'

wd = webdriver.Chrome('chromedriver', options=chrome_options)
wd.get(url)

movie_chart = wd.find_element(By.CLASS_NAME, 'sect-movie-chart')
contents = movie_chart.find_elements(By.CLASS_NAME, 'box-contents')
for content in contents:
    link = content.find_element(By.TAG_NAME, 'a').get_attribute('href') #selenium의 href 가져오는 코드
    title = content.find_element(By.CLASS_NAME, 'title').text
    percent = content.find_element(By.CLASS_NAME, 'percent').text
    info = content.find_element(By.CLASS_NAME, 'txt-info').text
    
#     print(title, percent, info, link)
    
    print(get_movie_reveiws(link, 2))
    
    
    


      Writer                            Review        Date
0    do**a45  스필버그의 꿈, 나도 같이 황홀했던 로미오와 줄리엣의 비극  2022.01.02
1        s덩h              결국 로미오와 줄리엣 스필버그엔 실망  2021.12.29
2  ch**o2524                    영화 웨스트 사이드 스토리  2021.12.28
3       쉐도우닉                   스티븐 스필버그도 끝물인가?  2021.12.28
      Writer                                             Review        Date
0        으네스                             기대 이상이였습니다 마지막 뭔데..ㅇㅅㅎ  2022.01.11
1     메로메로메롱                           와 주인공 운전실력에 무릎을 탁 치고 갑니다  2022.01.11
2     yy**nm                            특송 생각보다 재미있고 볼만합니당!! ㅎㅎ  2022.01.11
3       chdl  속도감 넘치는 전개와 카체이싱. 정은하와 함께 퀘스트를 통과해 내는 재미까지. 연출...  2022.01.11
4       아몰랑2                                    그냥 볼만합니다. 그냥그래요  2022.01.11
5    hh**674                                 액션이 가미되서 지루하지 않았어요  2022.01.11
6     이재현바라기                                  너무 재밌다고 합니다...ㅎㅎㅎ  2022.01.11
7   sos88426                                      생각보다 액션이 괜찮네요  2022.01.11
8      SinOf         

          Writer                                             Review  \
0           뺑덕아씨                             잔인하기만 하고 큰 재미나 감동은 없네요   
1       IMAXlike                                 기대안했는데 생각보다 재밌었습니다   
2           호랑지기                                       재미 있었습니다.^~^   
3      Onewish07   좋았어요! 기존 킹스맨의 느낌과 다르긴 했지만 프리퀄로서는 훌륭한 영화라고 생각합니다!   
4          영화광대역                           전편보단 못하지만 킬링 타임용으론 괜찮은 듯   
5         ki**45                                         너무너무너무재밌어용   
6            씔루기                액션이 시원하고 스타일리쉬합니다 킹스맨보던 분이라면 쭉 가야죠~   
7   ch**ujin2052  평이 안 좋아서 기대 안 했는데 생각보다 재밌었다! 액션이 대박 멋있고 재밌었다. ...   
8   ji**itsu0225                               킹스맨 시리즈이기보다 새로운 영화같음   
9          NaOtm        시대를 너무 역행한건지??? 다양한 눈요기감과 스토리전개가 좋았던 영화였어요.   
10           인진우     킹스맨 창설에 대한 이유를 담고 특유의 세련됨과 재미를 담을 수 있어서 좋았어요 ~   
11      Amethyst                                     좋았어요~^^좋았어요~^^   

          Date  
0   2022.01.11  
1   2022.01.11  
2   2022.01.11  
3   2022